<a href="https://colab.research.google.com/github/Mekondjo-EDSA/London_Bicycle_Project/blob/main/Twist_challenge_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OVERVIEW 
This notebook shows how to use BigQuery to train a model using SQL

## SETUP


Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager)
2. [Enable the BigQuery Storage API](https://cloud.google.com/bigquery/docs/reference/storage/#enabling_the_api)
3. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.


import required modules

In [ ]:

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from google.cloud import bigquery

Install required Packages, and restart runtime

In [ ]:
!pip install pip --upgrade 'google-cloud-bigquery[bqstorage,pandas]'


In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%load_ext google.colab.data_table

Authenticate 

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Set project ID and list public datasets available on BigQuery

In [ ]:
#variables
project_id = 'bigquery-public-data'
client = bigquery.Client(project=project_id)
for dataset in client.list_datasets():
  print(dataset.dataset_id)

Run some queries to get a feel for the data and how to structure the code

In [ ]:
%%bigquery --project twist-project-343915

SELECT
start_station_id, start_station_name, round(AVG(duration),2)duration
FROM `bigquery-public-data.london_bicycles.cycle_hire`
group by 1,2

In [ ]:
%%bigquery --project twist-project-343915

SELECT
EXTRACT(dayofweek from start_date)dayofweek, round(AVG(duration),2)duration
FROM `bigquery-public-data.london_bicycles.cycle_hire`
group by 1

In [ ]:
%%bigquery --project twist-project-343915

SELECT
EXTRACT(hour from start_date)hourofday, round(AVG(duration),2)duration
FROM `bigquery-public-data.london_bicycles.cycle_hire`
group by 1

In [ ]:
%%bigquery --project twist-project-343915

SELECT
bikes_count, AVG(duration) as duration
FROM 
  `bigquery-public-data`.london_bicycles.cycle_hire 
JOIN 
  `bigquery-public-data`.london_bicycles.cycle_stations
ON 
  cycle_hire.start_station_name = cycle_stations.name
group by 1

## Create dataset
The following section of code should be run in your Bigquery project in order to create the dataset and model.

In [ ]:
#create dataset 
%%bigquery --project twist-project-343915

SELECT
   EXTRACT(DATE from start_date) AS date,
   COUNT(*) AS num_trips
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
GROUP BY date

## Create model

In [ ]:
#create and train model
%%bigquery --project twist-project-343915

CREATE OR REPLACE MODEL forcasting_project.london_bike_arima_model
OPTIONS
  (model_type = 'ARIMA_PLUS', #create ARIMA time series model
   time_series_timestamp_col = 'date',
   time_series_data_col = 'num_trips'
  ) AS
SELECT
   EXTRACT(DATE from start_date) AS date,
   COUNT(*) AS num_trips
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
GROUP BY date

## Forecast the time series
Evaluation of the model can be performed in BigQuery or in Colab

In [ ]:
#Evaluate model
%%bigquery --project twist-project-343915

SELECT
  *
FROM
  ML.EXPLAIN_FORECAST(MODEL forcasting_project.london_bike_arima_model,
                      STRUCT(365 AS horizon, 0.9 AS confidence_level))

## Create model to forecast multiple time series simultaneously

In [ ]:
%%bigquery --project twist-project-343915

CREATE OR REPLACE MODEL forcasting_project.london_bike_arima_model_group
OPTIONS
  (model_type = 'ARIMA_PLUS',
   time_series_timestamp_col = 'date',
   time_series_data_col = 'num_trips',
   time_series_id_col = 'start_station_name',
   auto_arima_max_order = 5
  ) AS
SELECT
   start_station_name,
   EXTRACT(DATE from start_date) AS date,
   COUNT(*) AS num_trips
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
WHERE start_station_name LIKE '%Hyde Park%'
GROUP BY start_station_name, date

## Forecasting multiple time series simultaneously

In [ ]:
%%bigquery --project twist-project-343915
SELECT
 *
FROM
 ML.FORECAST(MODEL forcasting_project.london_bike_arima_model_group,
                     STRUCT(3 AS horizon, 0.9 AS confidence_level))